In [13]:
import pandas as pd
import numpy as np
from sshtunnel import SSHTunnelForwarder
import pymongo
import pprint
import pandas as pd
#from mongodb import client
from treeParsing import PatternTreeParsing
from graphviz import Digraph
import networkx as nx
import networkx.algorithms.isomorphism as iso

In [2]:
#change it to you way of get transaction frame
ethereum = client["ethereum"]
transaction=ethereum["transaction"]

In [3]:
#get your TX hash
id="0x2cf98dd2609ffe224ce81d4f60a759b36f868aca757592653d4abff2abdcd61a" 

In [4]:
# change it to your api to get the call frame
samples=transaction.find({'_id':id})
df_notes=pd.DataFrame(samples)
df_notes['stack'][0]
frame_list=df_notes['stack'][0]

In [5]:
# keep this if you are analysis ethereum data
platformToken='0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'.lower()

In [6]:
pattern_file=pd.read_csv('./pattern.csv')
pattern_list=pattern_file['pattern']

In [7]:
# calling PatternTreeParsing Class
patternTree=PatternTreeParsing(frame_list,platformToken,pattern_list)
# get parsing tree
list_of_parsing=patternTree.list_of_parsing
graph=patternTree.graph
# get pruning tree
simplify_tree=patternTree.simplify_tree
# get patttern tree
pattern_tree=patternTree.pattern_tree
#get discriptive pattern tree
discriptive_pattern_tree=patternTree.discriptive_pattern_tree

In [8]:
#Check Parsing Tree
discriptive_pattern_tree

[{'node': 'transfer_T0_1_2/burn_T3_2/transfer_T4_5_2/transfer_T4_2_1',
  'degree': '0-0',
  'address_mapping': {'Burn': 'Burn',
   'Mint': 'Mint',
   '5': '0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7',
   '2': '0xed279fdd11ca84beef15af5d39bb4d4bee23f0ca',
   '1': '0xd8c07491caa1edf960db3ceff387426d53942ea0'},
  'token_mapping': {'T4': ['0x6b175474e89094c44da98b954eedeac495271d0f'],
   'T3': ['0x6c3f90f043a72fa612cbac8115ee7e52bde6e490'],
   'T0': ['0x5f98805a4e8be255a32880fdec7f6728c6568ba0']},
  'nft_mapping': {}},
 {'node': 'transfer_T0_1_2/transfer_T3_2_1',
  'degree': '0-1',
  'address_mapping': {'Burn': 'Burn',
   'Mint': 'Mint',
   '1': '0xd8c07491caa1edf960db3ceff387426d53942ea0',
   '2': '0xba12222222228d8ba445958a75a0704d566bf2c8'},
  'token_mapping': {'T3': ['0x5f98805a4e8be255a32880fdec7f6728c6568ba0'],
   'T0': ['0x6b175474e89094c44da98b954eedeac495271d0f']},
  'nft_mapping': {}},
 {'node': 'empty', 'degree': '0'}]

In [9]:
# visualize parsing tree
g1 = Digraph('G', filename="parsing_tree"+id+".gv")
for i in range(len(list_of_parsing)):
    if list_of_parsing[i]['type']=="empty":
        g1.node(str(i), label='empty')
        
    elif list_of_parsing[i]['type']=="mint":
        string="mint \n"+"token: "+list_of_parsing[i]['token']+"\n to: "+list_of_parsing[i]['to']
        if(list_of_parsing[i]['tokenID']== 'empty'): 
            string="mint\n "+"token: "+list_of_parsing[i]['token']+"\n to: "+list_of_parsing[i]['to']
            g1.node(str(i), label=string)
        else:
            string="mint\n "+"nft token: "+list_of_parsing[i]['token']+", "+str(list_of_parsing[i]['tokenID'])+"\n to:"+list_of_parsing[i]['to']
            g1.node(str(i), label=string)  
            
    elif list_of_parsing[i]['type']=="burn":
        if(list_of_parsing[i]['tokenID']== 'empty'): 
            g1.node(str(i), label="burn\n "+"token: "+list_of_parsing[i]['token']+"\n from:"+list_of_parsing[i]['from'])
        else:
            g1.node(str(i), label="burn \n "+"nft token: "+list_of_parsing[i]['token']+" "+str(list_of_parsing[i]['tokenID'])+"\n from:"+list_of_parsing[i]['from'])  
    else:
        if(list_of_parsing[i]['tokenID']== 'empty'): 
            g1.node(str(i), label="transfer\n "+"token: "+list_of_parsing[i]['token']+"\n from: "+list_of_parsing[i]['from']+"\n to:"+list_of_parsing[i]['to'])
        else:
            g1.node(str(i), label="transfer \n"+"nft token: "+list_of_parsing[i]['token']+" "+str(list_of_parsing[i]['tokenID'])+"\n from:"+list_of_parsing[i]['from']+"\n to:"+list_of_parsing[i]['to'])  
for j in  graph.keys():
    for k in graph[j]:
        g1.edge(str(j), str(k))

In [10]:
# visualize pruning tree
effective_node=patternTree.get_effective_nodes(simplify_tree)
g2 = Digraph('G', filename="pruning tree"+id+".gv")
for i in range(len(list_of_parsing)):
    if i not in effective_node:
        continue
    if list_of_parsing[i]['type']=="empty":
        g2.node(str(i), label='empty')
        
    elif list_of_parsing[i]['type']=="mint":
        string="mint \n"+"token: "+list_of_parsing[i]['token']+"\n to: "+list_of_parsing[i]['to']
        if(list_of_parsing[i]['tokenID']== 'empty'): 
            string="mint\n "+"token: "+list_of_parsing[i]['token']+"\n to: "+list_of_parsing[i]['to']
            g2.node(str(i), label=string)
        else:
            string="mint\n "+"nft token: "+list_of_parsing[i]['token']+", "+str(list_of_parsing[i]['tokenID'])+"\n to:"+list_of_parsing[i]['to']
            g2.node(str(i), label=string)  
            
    elif list_of_parsing[i]['type']=="burn":
        if(list_of_parsing[i]['tokenID']== 'empty'): 
            g2.node(str(i), label="burn\n "+"token: "+list_of_parsing[i]['token']+"\n from:"+list_of_parsing[i]['from'])
        else:
            g2.node(str(i), label="burn \n "+"nft token: "+list_of_parsing[i]['token']+" "+str(list_of_parsing[i]['tokenID'])+"\n from:"+list_of_parsing[i]['from'])  
    else:
        if(list_of_parsing[i]['tokenID']== 'empty'): 
            g2.node(str(i), label="transfer\n "+"token: "+list_of_parsing[i]['token']+"\n from: "+list_of_parsing[i]['from']+"\n to:"+list_of_parsing[i]['to'])
        else:
            g2.node(str(i), label="transfer \n"+"nft token: "+list_of_parsing[i]['token']+" "+str(list_of_parsing[i]['tokenID'])+"\n from:"+list_of_parsing[i]['from']+"\n to:"+list_of_parsing[i]['to'])  
for j in  simplify_tree.keys():
    for k in simplify_tree[j]:
        g2.edge(str(j), str(k))

In [11]:
# visualize pattern tree
g3 = Digraph('G', filename="pattern_tree"+id+".gv")
for i in range(len(pattern_tree)):
    g3.node(pattern_tree[i]['degree'], label=pattern_tree[i]['node'])
for i in range(len(pattern_tree)):
    if pattern_tree[i]['degree']!='0':  
        father_node=pattern_tree[i]['degree'][:-2]
        g3.edge(father_node, pattern_tree[i]['degree'])

In [12]:
g1.view()
g2.view()
g3.view()

'pattern_tree0x2cf98dd2609ffe224ce81d4f60a759b36f868aca757592653d4abff2abdcd61a.gv.pdf'